In [1]:
# 求職網非同步爬蟲

In [1]:
import requests as rq
from pyquery import PyQuery as pq
import re
import time
from datetime import datetime
import pandas as pd
import sqlalchemy as sa
from sqlite3 import OperationalError

from aiohttp import ClientSession
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [24]:
async def main(web):
    global df, keyword, end_page
    df = pd.DataFrame()
    async with ClientSession() as session:
        if web=='104':
            urls = [f'https://www.104.com.tw/jobs/search/?keyword={keyword}&order=1&page={str(i)}&jobsource=2018indexpoc&ro=0' for i in range(1,end_page+1)]
            tasks = [asyncio.create_task(fetch_104(url, session)) for url in urls]  # 建立任務清單
        if web=='cakeresume':
            urls = [f'https://www.cakeresume.com/jobs?q={keyword}&ref=navbar_quick_link_jobs&page={str(i)}' for i in range(1,end_page+1)]
            tasks = [asyncio.create_task(fetch_cakeresume(url, session)) for url in urls]  # 建立任務清單
        if web=='1111':
            urls = [f'https://www.1111.com.tw/search/job?ks={keyword}&fs=1&page={str(i)}' for i in range(1,end_page+1)]
            tasks = [asyncio.create_task(fetch_1111(url, session)) for url in urls]  # 建立任務清單
        await asyncio.gather(*tasks)  # 打包任務清單及執行

async def fetch_104(url, session):
    global df
    date, title, company, salary, link = [], [], [], [], []
    async with session.get(url) as response:  #非同步發送請求
        text = await response.text()
        doc = pq(text)
        for each in doc('.b-block__left').items():
            if each('.b-tit a'):
                if each('.b-tit__date').text():
                    date.append(each('.b-tit__date').text())
                else:
                    date.append('-')
                title.append(each('.b-tit .js-job-link').text())
                company.append(each('.b-list-inline.b-clearfix a').text())
                salary.append(each('.b-tag--default:nth-child(1)').text())
                link.append('https:'+each('.b-tit .js-job-link').attr("href"))
        
        crawl_at = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
        data_dict = dict(zip(['刊登日期','職稱','公司名稱','薪資','webURL','關鍵字','爬蟲時間'],
                             [date,title,company,salary,link,keyword,crawl_at]))
        df = df.append(pd.DataFrame(data_dict))
        return df
    
async def fetch_cakeresume(url, session):
    global df
    title, link, company, job_desc, date, level, location, salary, tags = [],[],[],[],[],[],[],[],[]
    async with session.get(url) as response:  #非同步發送請求
        text = await response.text()
        doc = pq(text)
        for each in doc('.job.well-list-item.well-list-item-link').items():
            title.append(each('.job-title').text())
            link.append(each('.job-link').attr("href"))
            company.append(each('.page-name a').text())
            job_desc.append(each('.job-desc').text().replace('\n',''))
            date.append(each('.update-section').text())
            for item in each('.job-list-item-tags').items():
                level.append(item('.refinement-tags a').text())
                location.append(item('.info-section .location-section').text())
                salary.append(item('.info-section .job-salary-section').text())
                tags.append(item('.label').text())
        
        crawl_at = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
        data_dict = dict(zip(['刊登日期','職稱','階級','描述','公司名稱','地點','薪資','webURL','關鍵字','爬蟲時間'],
                             [date,title,level,job_desc,company,location,salary,link,keyword,crawl_at]))
        df = df.append(pd.DataFrame(data_dict))
        return df
    
async def fetch_1111(url, session):
    global df
    title,date,company,department,location,salary,experience,education,job_desc,link = [],[],[],[],[],[],[],[],[],[]
    async with session.get(url) as response:  #非同步發送請求
        text = await response.text()
        doc = pq(text)
        for each in doc('.row.no-gutters.si-item.si-1.digest').items():
            title.append(each('.text-truncate.position0Link.mobileItemClick').text())
            link.append(each('.text-truncate.position0Link.mobileItemClick').attr("href"))
            company.append(each('.d-block.d-md-none.text-truncate.jb-organ-m').text())
            date.append(each('.date').text().replace('更新時間：',''))
            department.append(each('.item-job__desc-limit-item').text())
            job_desc.append(each('.col-12.jbInfoTxt.UnExtension').text())
            location.append(each('.text-truncate.needs span:nth-child(1)').text())
            salary.append(each('.text-truncate.needs span:nth-child(1)').text())
            experience.append(each('.text-truncate.needs span:nth-child(3)').text())
            education.append(each('.text-truncate.needs span:nth-child(3)').text())

        crawl_at = datetime.now().strftime('%Y/%m/%d %H:%M:%S')
        data_dict = dict(zip(['刊登日期','職稱','描述','經驗','學歷','科系','公司名稱','地點','薪資','webURL','關鍵字','爬蟲時間'],
                             [date,title,job_desc,experience,education,department,company,location,salary,link,keyword,crawl_at]))
        df = df.append(pd.DataFrame(data_dict))
        return df
    
# 存檔
def save_to_sqlite(web, engine):
    # 讀sqlite裏頭的資料表名稱
    tables = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{web}'", engine)
    if tables.empty:
        df_raw = pd.DataFrame()
    else:
        df_raw = pd.read_sql(web, engine)
    df_raw = df_raw.append(df)
    df_raw.drop_duplicates(subset=['職稱','公司名稱'],keep='last',inplace=True)
    df_raw.to_sql(web, engine, if_exists="replace", index=False)
    print('存檔成功')
    
# 執行爬蟲
def start_crawling():
    global sqlite_path, web
    engine = sa.create_engine(f"sqlite:///{sqlite_path}")
    loop = asyncio.get_event_loop()  #建立事件迴圈(Event Loop)
    start = time.time()
    loop.run_until_complete(main(web))  #執行協程(coroutine)
    stop = time.time()
    print(f"歷經{stop-start}秒")
    save_to_sqlite(web, engine)

In [25]:
web = '1111'
# '104'
# 'cakeresume'
# '1111'
keyword = 'python'
end_page = 3
sqlite_path = './job.db'

In [30]:
start_crawling()

歷經2.1826727390289307秒
存檔成功


In [31]:
df

,刊登日期,職稱,描述,經驗,學歷,科系,公司名稱,地點,薪資,webURL,關鍵字,爬蟲時間
0,2020-12-21,後端開發工程師,*對Python編碼熟悉\n*對Python和JavaScript有著深刻的理解\n*數據庫...,1年工作經驗以上,1年工作經驗以上,,協雲科技股份有限公司,台中市西屯區,台中市西屯區,https://www.1111.com.tw/job/91191473/,python,2020/12/25 15:17:03
1,2020-12-21,自動化工程師,需有\n軟體 網頁 相關測試經驗\n熟悉程式語言： python\n熟悉測試工具：Jmeter,1年工作經驗以上,1年工作經驗以上,,德義資訊股份有限公司,台北市大安區,台北市大安區,https://www.1111.com.tw/job/76812064/,python,2020/12/25 15:17:03
2,2020-12-23,資料工程師(T53),1.Hadoop-based應用系統開發維護及專案支援。\n2.巨量資料處理及ETL架構規劃...,2年工作經驗以上,2年工作經驗以上,"資訊管理學類,資訊工程學類,數學統計學門",關貿網路股份有限公司,台北市南港區,台北市南港區,https://www.1111.com.tw/job/91671907/,python,2020/12/25 15:17:03
3,2020-12-23,DevOps工程師(T61),1. OpenStack / Ceph / Kubernetes 設計及維護經驗。\n2. ...,1年工作經驗以上,1年工作經驗以上,,關貿網路股份有限公司,台北市南港區,台北市南港區,https://www.1111.com.tw/job/91671819/,python,2020/12/25 15:17:03
4,2020-12-21,軟體測試工程師 _ 汐止 _ 00000,1. 測試計畫與測試項目的安排規劃與撰寫\n2. 對於NB軟體進行整合測試/功能測試\n3....,經驗不拘,經驗不拘,,緯創資通股份有限公司(緯創資通),新北市汐止區,新北市汐止區,https://www.1111.com.tw/job/91436459/,python,2020/12/25 15:17:03
5,2020-12-21,RRS2000015-系統開發工程師,"軟體系統開發,主要程式語言為python,以django為框架,使用Azure DevOps...",經驗不拘,經驗不拘,,緯創資通股份有限公司(緯創資通),新北市汐止區,新北市汐止區,https://www.1111.com.tw/job/91294827/,python,2020/12/25 15:17:03
6,2020-12-19,系統軟體開發工程師,1. 具程式設計開發經驗.\n2. 熟 C、C++、Java、Python、MySQL、 l...,經驗不拘,經驗不拘,,捷螺系統股份有限公司,新竹縣竹北市,新竹縣竹北市,https://www.1111.com.tw/job/92161189/,python,2020/12/25 15:17:03
7,2020-12-25,I3501 程式設計師/前台網頁開發師,"– 建置及開發 ITIL 系統平台(CMDB, TT, Change/Release), 整...",1年工作經驗以上,1年工作經驗以上,,遠傳電信股份有限公司,新北市板橋區,新北市板橋區,https://www.1111.com.tw/job/92134069/,python,2020/12/25 15:17:03
8,2020-12-22,人工智慧醫療創新發展中心-軟體工程師,資格條件:\n大學或碩士，工作經驗2年以上\n專長技能:\nPython3\n- Exper...,2年工作經驗以上,2年工作經驗以上,"系統設計學類,軟體發展學類,網路學類",佛教慈濟醫療財團法人花蓮慈濟醫院,花蓮縣花蓮市,花蓮縣花蓮市,https://www.1111.com.tw/job/76834054/,python,2020/12/25 15:17:03
9,2020-12-25,Switch SQA工程師_內湖/新竹,1、 網路通訊協定(Layer2/Layer3)之研究\n2、 驗證系統功能、壓力測試與網路...,5年工作經驗以上,5年工作經驗以上,"資訊工程學類 CCNA,CCNP,CCIE",緯創資通股份有限公司(緯創資通),台北市內湖區,台北市內湖區,https://www.1111.com.tw/job/92168178/,python,2020/12/25 15:17:03


In [32]:
df[df.duplicated()]

,刊登日期,職稱,描述,經驗,學歷,科系,公司名稱,地點,薪資,webURL,關鍵字,爬蟲時間
3,2018-03-24,運維工程師(3001208),職責要求\n1.負責LINUX、MYSQL、網路設備、安全設備、應用系統的的基本安裝與維護，...,2年工作經驗以上,2年工作經驗以上,,1111高階獵頭顧問中心,台北市內湖區,台北市內湖區,https://www.1111.com.tw/job/84823449/,python,2020/12/25 15:17:03
4,2020-12-24,110RD03(研發替代役)-AI演算法應用開發工程師,1.機器學習/深度學習/文字探勘演算法開發應用(R/Python)\n2.Open Data...,經驗不拘,經驗不拘,"資訊管理學類,資訊工程學類",財團法人金屬工業研究發展中心(金屬中心廣告),高雄市楠梓區,高雄市楠梓區,https://www.1111.com.tw/job/92129812/,python,2020/12/25 15:17:03
